In [27]:
import requests
import hashlib
import time
import pandas as pd
from pandas import json_normalize

secret_token = '696a6f47212d93f88cef71238cfb3d1cd357905b3e34eabd2bbc49124553c129'

# Generate the Unix timestamp
timestamp = int(time.time())

# Construct the query string
query_string = f'time={timestamp}'

# Generate the hash
hash_string = secret_token + str(timestamp)
hash_value = hashlib.sha512(hash_string.encode()).hexdigest()

# Construct the URL with the query string and hash
url = f'https://api.fabdb.net/cards?{query_string}&hash={hash_value}'

parameters = {
    "page": 1,
    "per_page": 100
}

total_pages = 21

# Set the request headers
headers = {
    'Authorization': f'Bearer {secret_token}',
    'Accept': 'application/json'
}


In [28]:
#empty list to hold paginated data
data_records = []

for page in range(1, total_pages + 1):
    parameters['page'] = page
    
    response = requests.get(url, params=parameters)

    # process response
    data = response.json()

    # Extract the relevant data records from the response
    data_records.extend(data['data'])

# Normalize the data before storing it in the dataframe
df = pd.json_normalize(data_records)


In [29]:
df.rename(columns={'stats.resource': 'pitch value'}, inplace=True)

# Map the values in 'pitch value' column to corresponding colors
df['pitch value'] = df['pitch value'].map({1: 'Red', 2: 'Yellow', 3: 'Blue'})

# Rename other columns
df.rename(columns={'stats.life': 'life', 'stats.intellect': 'intellect','stats.attack': 'attack', 'stats.defense': 'defense', 'stats.cost': 'cost'}, inplace=True)
#drop stats column after it's been split
columns_to_drop = ['stats']
df.drop(columns_to_drop, axis=1, inplace=True)


In [40]:
# converting select columns to numeric for statistical analysis
columns_to_convert = ['life', 'attack', 'defense', 'cost', 'intellect']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce', downcast='float')
pd.set_option('display.max_rows', None)  # Show all rows
df.head(10)

# for i in df.printings:
#     print(i)
#     print()

,identifier,name,legality,keywords,text,rarity,image,printings,pitch value,life,intellect,attack,defense,cost
0,eye-of-ophidia,Eye of Ophidia,[],"[generic, resource, gem]",**Legendary** *(You may only have 1 Eye of Oph...,F,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 863, 'language': 'en', 'name': 'Eye of...",Blue,NaN,NaN,NaN,NaN,NaN
1,dash-inventor-extraordinaire,"Dash, Inventor Extraordinaire",[],"[mechanologist, hero]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 864, 'language': 'en', 'name': 'Dash, ...",NaN,40.0,4.0,NaN,NaN,NaN
2,dash,Dash,[],"[mechanologist, hero, young]",You may start the game with a Mechanologist it...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 865, 'language': 'en', 'name': 'Dash',...",NaN,20.0,4.0,NaN,NaN,NaN
3,teklo-plasma-pistol,Teklo Plasma Pistol,[],"[mechanologist, weapon, pistol, 2h]",**Action** - Remove a steam counter from Teklo...,T,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 866, 'language': 'en', 'name': 'Teklo ...",NaN,NaN,NaN,2.0,NaN,NaN
4,teklo-foundry-heart,Teklo Foundry Heart,[],"[mechanologist, equipment, chest]",**Once per Turn Action** - [Resource]: Banish ...,L,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 867, 'language': 'en', 'name': 'Teklo ...",NaN,NaN,NaN,NaN,2.0,NaN
5,achilles-accelerator,Achilles Accelerator,[],"[mechanologist, equipment, legs]",**Instant** - Destroy Achilles Accelerator: Ga...,C,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 868, 'language': 'en', 'name': 'Achill...",NaN,NaN,NaN,NaN,0.0,NaN
6,high-octane,High Octane,[],"[mechanologist, action]","Whenever you **boost** a card this turn, gain ...",M,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 870, 'language': 'en', 'name': 'High O...",Red,NaN,NaN,NaN,3.0,1.0
7,teklo-core,Teklo Core,[],"[mechanologist, action, item]",**Dash specialization** *(You may only have Te...,M,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 872, 'language': 'en', 'name': 'Teklo ...",Blue,NaN,NaN,NaN,NaN,0.0
8,maximum-velocity,Maximum Velocity,[],"[mechanologist, action, attack]",Play Maximum Velocity only if you have **boost...,S,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 874, 'language': 'en', 'name': 'Maximu...",Red,NaN,NaN,10.0,3.0,2.0
9,spark-of-genius,Spark of Genius,[],"[mechanologist, action]",**Dash Specialization** *(You may only have Sp...,S,https://fabdb2.imgix.net/cards/printings/ARC00...,"[{'id': 876, 'language': 'en', 'name': 'Spark ...",Yellow,NaN,NaN,NaN,3.0,0.0


In [32]:
#avg numeric stats for all cards
avg_stats = df[['life', 'attack', 'defense','intellect', 'cost']].mean(axis=0,numeric_only=True,skipna=True)
print(avg_stats)

life         22.194805
attack        3.907025
defense       2.524713
intellect     3.983607
cost          1.171671
dtype: float32


In [33]:
# a filtered df showing only Legendary and Mystical Cards with Pitch Value of 3
filtered_df = df[(df['pitch value'] == 'Blue') & ((df['rarity'] == 'M') | (df['rarity'] == 'L'))]

#printing second and third columns and only first 4 rows.
print("Seond and Third Columns/4 rows:")
print(filtered_df.head(4)[['name', 'legality']])


Seond and Third Columns/4 rows:
                    name legality
7             Teklo Core       []
245      Stamp Authority       []
266          Find Center       []
310  Meganetic Shockwave       []


In [34]:
#total nan values for selected columns in blue cards with L or M rarity
nan_counts_blue = filtered_df[['life', 'attack', 'defense', 'cost', 'intellect']].isna().sum()
print(nan_counts)

NameError: name 'nan_counts' is not defined

In [ ]:
#getting same avg numeric stats for only blue cards that are Legendary or Mystical that pitch for 3
blue_card_stats = filtered_df[['life', 'attack', 'defense','intellect', 'cost']].mean(axis=0,numeric_only=True,skipna=True)
print(blue_card_stats)


life              NaN
attack       2.307692
defense      2.770833
intellect         NaN
cost         1.014706
dtype: float32
